In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
mpl.rcParams['savefig.dpi'] = 128
mpl.rcParams['figure.dpi'] = 128
# Plot size to 14" x 7"
mpl.rc('figure', figsize = (14, 7))
# Font size to 14
mpl.rc('font', size = 14)
# Do not display top and right frame lines
mpl.rc('axes.spines', top = False, right = False)
# Remove grid lines
mpl.rc('axes', grid = False)
# Set backgound color to white
mpl.rc('axes', facecolor = 'white')

In [2]:
# read whole year data
allFiles = glob.glob("data/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,encoding="utf-8")
    list_.append(df)
df = pd.concat(list_)

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
def transform_cols(df):
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('_', ' ')
    return df

In [5]:
df = transform_cols(df)

In [6]:
# selected_features = ['marketing code','suburb', 'state','post code','enquired',
#                      'loan amount','loan reason','property use']
# selected_features = ['marketing code','enquired',
#                      'loan amount','loan reason','property use']
selected_features = ['marketing code','enquired','loan amount']
target = 'enquiry status';
whole_set = selected_features + [target]

df = df[whole_set]

In [7]:
df = df[~df[target].isin(['In Progress','Just Received','On Hold'])]
df.shape

(88168, 4)

In [8]:
df['loan amount'].fillna(0,inplace=True)

In [9]:
df['loan amount'] = df['loan amount'].str.replace(",","")

In [10]:
invalid_columns= ['500001-$1000000',
                  '300001-$500000',
                  '0-$300000',
                  '250000 - 300000',
                  '250000-350000',
                  '2600 monthly',
                  'not_sure',
                  '1000,001+',
                 '9999-',
                  'I50000',
                  '1.5 M',
                  '1000001+',
                  '9999-',
                  '80-90k']

In [11]:
df = df[~df['loan amount'].isin(invalid_columns)]
df.shape

(87716, 4)

In [12]:
df['loan amount'] = df['loan amount'].str.replace(".00","")
df

,marketing code,enquired,loan amount,enquiry status
0,A12668,20-Jun-2014 14:32:10,370,Accepted
1,A9301,23-Sep-2014 13:20:48,260,Accepted
2,A12667,22-Dec-2014 15:22:17,618920,Accepted
3,A9263,15-Feb-2015 20:08:55,490,Accepted
4,P0001,13-Apr-2015 16:22:35,600,Rejected
5,A8887,03-Jun-2015 09:14:52,273485,Accepted
6,A9326,22-Jun-2015 22:12:23,305862,Accepted
7,A9326,03-Jul-2015 23:56:55,,Rejected
8,P0001,10-Jul-2015 22:46:25,100,Rejected
9,A119,15-Jul-2015 20:08:11,265248,Accepted


In [13]:
for _ in df.columns:
    print("The number of null values in:{} == {}".format(_, df[_].isnull().sum()))

The number of null values in:marketing code == 31
The number of null values in:enquired == 0
The number of null values in:loan amount == 31499
The number of null values in:enquiry status == 0


In [14]:
# df.dropna(axis=0, how='any')
df = df[~df['marketing code'].isnull()]

In [15]:
for _ in df.columns:
    print("The number of null values in:{} == {}".format(_, df[_].isnull().sum()))
    
df.shape

The number of null values in:marketing code == 0
The number of null values in:enquired == 0
The number of null values in:loan amount == 31490
The number of null values in:enquiry status == 0


(87685, 4)

In [16]:
def transform(df): 
    df['loan amount'] = df['loan amount'].astype('float')
    df['enquired'] = pd.DatetimeIndex(df['enquired'])
    df['month'] = df['enquired'].dt.month
    df['day'] = df['enquired'].dt.day
    df['hour'] = df['enquired'].dt.hour
    df['weekday'] = df['enquired'].dt.weekday_name
    
    if 'enquired'in df.columns:
        df.drop(['enquired'], axis = 1, inplace = True)
    

In [17]:
transform(df)

ValueError: could not convert string to float: 

In [ ]:
print (df[ pd.to_numeric(df['loan amount'], errors='coerce').isnull()])

In [ ]:
print("Shape of data frame: {}".format(df.shape))
print("Keys of enquiries_dataset: \n{}".format(df.keys()))
print("data ytpes of enquiries_dataset: \n{}".format(df.dtypes))

In [ ]:
# # set all null loan amount to mean valu
# df['loan amount'] = df['loan amount'].fillna(df['loan amount'].mean())
# for _ in df.columns:
#     print("The number of null values in:{} == {}".format(_, df[_].isnull().sum()))
# test = df[df['loan amount'].isnull()]
# result = test.loc[test['enquiry status'] == 'Accepted']
# result

# test = df[df['enquiry status'] == 'Accepted']
# test['marketing code'].value_counts()

In [ ]:
test = df[df['enquiry status'] == 'Rejected']
test.shape

In [ ]:
test = df[df['enquiry status'] == 'Accepted']
test.shape

In [ ]:
print("Shape of new data frame: {}".format(df.shape))

In [ ]:
print("data ytpes of enquiries_dataset: \n{}".format(df.dtypes))

In [ ]:
from sklearn.preprocessing import LabelEncoder

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col].astype(str))
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
# df = df.iloc[:number_of_rows]
X = df[df.keys()]
X = df.loc[:,df.columns != target]

le = LabelEncoder()
y = df[target]
y = le.fit_transform(y)
le.classes_

In [ ]:
category_column = X.select_dtypes(include='object')

In [ ]:
# X = MultiColumnLabelEncoder(columns = category_column.columns).fit_transform(X)
X = pd.get_dummies(X, columns=category_column.columns)
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y,
                     test_size=0.3,
                     random_state=0,
                     stratify=y)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipe_rf = make_pipeline(StandardScaler(),
                        RandomForestClassifier(n_estimators=500,random_state=1))

pipe_rf.fit(X_train, y_train)
rf_label = pipe_rf.predict(X_test)
# print('Test Accuracy: %.3f' % pipe_rf.score(X_test, y_test))

In [ ]:
# train_sizes, train_scores, test_scores = learning_curve(estimator=pipe_rf,
#                                                         X=X_train,y=y_train,train_sizes=np.linspace(0.1, 1.0, 10),
#                                                         cv=10,n_jobs=1)
# train_mean = np.mean(train_scores,axis=1)
# train_std = np.std(train_scores, axis=1)      
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)


# plt.plot(train_sizes, train_mean,color='blue', marker='o',markersize=5,label='training accuracy')
# plt.fill_between(train_sizes,train_mean + train_std,train_mean - train_std,alpha=0.15, color='blue')
# plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
# plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
# plt.grid()
# plt.xlabel('Number of training samples')
# plt.ylabel('Accuracy')               
# plt.legend(loc='lower right')
# plt.ylim([0.8, 1.0])
# plt.show()

In [ ]:
from sklearn import cross_validation
from sklearn.metrics import classification_report

# scores = cross_validation.cross_val_score(pipe_lr, X, y, cv=5)
# print("LogisticRegression cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

# print("LogisticRegression")
# print(classification_report(y_test, y_pred))

scores = cross_validation.cross_val_score(pipe_rf, X, y, cv=5)
print("Random forest cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

print("Random forest")
print(classification_report(y_test, rf_label))

# scores = cross_validation.cross_val_score(pipe_svm, X, y, cv=5)
# print("SVM cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

# print("SVM")
# print(classification_report(y_test, svm_pred))

# scores = cross_validation.cross_val_score(pipe_dt, X, y, cv=5)
# print("Decision Tree cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

# print("Decision Tree")
# print(classification_report(y_test, dt_label))

# scores = cross_validation.cross_val_score(pipe_gnb, X, y, cv=5)
# print("GaussianNB cross_validation: {:.2f}".format(np.mean(scores, axis=0)))

# print("GaussianNB")
# print(classification_report(y_test, gnb_label))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# from sklearn import metrics

# # rescale data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# rfc = RandomForestClassifier(n_estimators=500,random_state=1)
# rfc.fit(X_train_scaled, y_train)
# pred_forest = rfc.predict(X_test)

# print("Test score: {:.2f}".format(rfc.score(X_test_scaled, y_test)))


In [ ]:
# train_sizes, train_scores, test_scores = learning_curve(estimator=rfc,
#                                                         X=X_train,y=y_train,train_sizes=np.linspace(0.1, 1.0, 10),
#                                                         cv=10,n_jobs=1)
# train_mean = np.mean(train_scores,axis=1)
# train_std = np.std(train_scores, axis=1)      
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)


# plt.plot(train_sizes, train_mean,color='blue', marker='o',markersize=5,label='training accuracy')
# plt.fill_between(train_sizes,train_mean + train_std,train_mean - train_std,alpha=0.15, color='blue')
# plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
# plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
# plt.grid()
# plt.xlabel('Number of training samples')
# plt.ylabel('Accuracy')               
# plt.legend(loc='lower right')
# plt.ylim([0.8, 1.0])
# plt.show()

In [ ]:
# feat_labels = X.columns[0:]
# importances = rfc.feature_importances_
# importances = [x for x in j if x >= 5]

# # reverse the list
# indices = np.argsort(importances)[::-1]

# for f in range(X_train.shape[1]):
#     print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]],importances[indices[f]]))
    
# plt.title('Feature Importance')
# plt.bar(range(X_train.shape[1]),importances[indices],align='center')
# plt.xticks(range(X_train.shape[1]),feat_labels[indices], rotation=90)
# plt.xlim([-1, X_train.shape[1]])
# plt.tight_layout()
# plt.show()

In [ ]:
# import pickle
# import os

# dest = os.path.join('movieclassifier', 'pkl_objects')
# if not os.path.exists(dest):
#     os.makedirs(dest)

# pickle.dump(stop,open(os.path.join(dest, 'stopwords.pkl'),'wb'),protocol=4)
# pickle.dump(clf,
            
# from sklearn.externals import joblib
# joblib.dump(pipe_rf, 'app/models/classifier.pkl')

In [ ]:
# model_columns = list(X.columns)
# joblib.dump(model_columns, 'app/models/model_columns.pkl')

In [ ]:
clf = joblib.load('app/models/classifier.pkl')
model_columns = joblib.load('app/models/model_columns.pkl')

In [ ]:
import json
my_json_string = json.dumps({
    "marketing_code": "CBADOM16",
    "loan amount": "260000",
    "enquired":'27/1/17 19:21'
})
my_json_string

In [ ]:
# test = pd.read_json(my_json_string, orient='index')
# data = pd.read_json(my_json_string, typ='series',orient='index')
# data = pd.DataFrame(data=data)
data = json.loads(my_json_string)
data = pd.DataFrame(data,index=[0])

In [ ]:
data = transform(data)

In [ ]:
# y_pred = pipe_rf.predict(data)
# pipe_rf.predict_proba(data)
category_column.columns
data = pd.get_dummies(data)

In [ ]:
def cleanFeatures(data) :
    for col in data.columns:
        if col not in model_columns:
            data.drop([col], axis=1, inPlace=True)
            
    print(data.shape)     
    
    for col in model_columns: 
        print(col)
        if col not in data.columns:
            print('here')
            data[col] = 0
    print(len(model_columns))    
    print(data.shape)   

In [ ]:
cleanFeatures(data)

In [ ]:
prediction = clf.predict(data)

In [ ]:
prediction

In [ ]:
print("posibility is {}:".format(np.max(clf.predict_proba(data))))